# Querying Azure Sentinel via REST API using a Personal Access Token (PAT)

<table align="left" style="margin-left:25px">
  <tr>
    <th>Type</th>
    <th>Title</th>
    <th>Description</th>
  </tr>
  <tr>
      <td>Documentation</td>
      <td><b><a href="https://techcommunity.microsoft.com/t5/azure-sentinel/extending-azure-sentinel-apis-integration-and-management/ba-p/1116885">Extending Azure Sentinel: APIs, Integration and management automation</a></td>
    <td>Its highly recommended to follow this guide to get familiar with navigating and using Jupyter Notebooks. </td>
  </tr>
  <tr>
      <td>Tech Community</td>
    <td><b><a href="../../PowerShell/Getting-Started-README.ipynb">Access Azure Sentinel Log Analytics via API (Part 1)</a></td>
    <td>@rinure wrote a good article explaining how to query the API. Use this as a reference to understand how to query the Azure Sentinel API. </td>
  </tr>
</table> 

## Get Workspaces in Subscription

### Configure variables. Change this based on your tenant, domain, and app registration details

In [9]:
<#
		`$config = @{
			tenantId = $tenantID                # This is the tenant ID
			appId = $appId                      # This is the client ID when you register your app
			appSecret = $appSecret              # This is the generated client secret in the client app
            sentinelWorkspaceID = "afadfafafaf" # Log analytics workspace sentinel is using
		}
<#

## Import credential from storage path or Export a new XML file using above filled in parameters

In [2]:
## Checks to see if config file exist and if not export credential file from above.
$storagePath = ".\AzSentinel.credential"
$config = (Import-CliXml -Path $storagePath)
if (!$config) {
    $config | Export-CliXml -Path $storagePath
} else {
    Write-Host -Foregroundcolor green "`nCredential file loaded from $($storagePath)"
}


Credential file loaded from .\AzSentinel.credential


## Authenticate with the MS Graph API via Personal Access Token (PAT)

In [ ]:
Import-Module "..\..\Graph-API\modules\GraphAuthentication.psm1" -Force  ## Imports the module
#Get-Command -Module "GraphAuthentication"                  ## Shows what commands were imported

$resourceURI = "https://api.loganalytics.io"

## authenticate with the graph API via REST API
$aadToken = Invoke-GraphAuthenticatePAT -authvariables $config -resource $resourceURI
Write-Host ($aadToken -replace "\d","*")

### Put your Azure Sentinel Hunting Query here

In [5]:
## Enter your query here
$query = @"

SecurityAlert

"@

$timeSpan = "" ## You can leave blank or specify a date. But something HAS to be there. 
 
## This is the query converted to JSON format
$body = (@{
   query = $Query
   timespan = $timeSpan  
} | ConvertTO-JSON)
Write-Host -ForeGround Yellow "`nQuery"; $Query
Write-Host -ForeGround Yellow "JSON"; $body


Query

SecurityAlert

JSON
{
  "query": "\nSecurityAlert\n",
  "timespan": ""
}


## We will query the API

In [6]:
$URI = "$($resourceURI)/v1/workspaces/$($config.sentinelWorkspaceID)/query"
$header = @{ 
    'Content-Type' = 'application/json'
    Accept = 'application/json'
    Authorization = "Bearer $($aadToken)" 
}

Write-Host "URI: $($URI -replace "[0-9]","*")"
$header

URI: https://api.loganalytics.io/v*/workspaces/a*******-*bde-****-*f**-*c**a**cc***/query

Name                           Value
----                           -----
Authorization                  Bearer eyJ0eXAiOiJKV1QiLCJhbGciOiJSUzI1NiIsIng1dCI6IlNzWnNCTmhaY0Yz…
Accept                         application/json
Content-Type                   application/json



## Query Graph API and insert into $graphresponse variable

In [8]:
$Result = (Invoke-RestMethod -Method POST -Uri $URI -Headers $header -body $body -ErrorAction Stop)
$Result.tables.rows | Select -First 1

a8960181-0bde-4955-9f94-2c12a36cc900

Tuesday, June 2, 2020 7:43:13 PM
DLP - Sharepoint - Financial sensitive file(s) shared externally
DLP - Sharepoint - Financial sensitive file(s) shared externally
High
File policy 'DLP - Sharepoint - Financial sensitive file(s) shared externally' was matched by 'Credit_Card_Form_SP_20200602-T0239.docx'
MCAS
Microsoft
457088D9-F5CC-3065-8A4F-055C7A53D39A
e07f746e-69f7-fafa-e454-9a3270364fc1


MCAS_ALERT_CABINET_EVENT_MATCH_FILE
Unknown

False
Tuesday, June 2, 2020 7:42:59 PM
Tuesday, June 2, 2020 7:42:59 PM
Tuesday, June 2, 2020 7:43:13 PM

{
  "Cloud Applications": "Microsoft SharePoint Online"
}
[
  {
    "$id": "3",
    "AppId": 20892,
    "Name": "Microsoft SharePoint Online",
    "InstanceName": "Microsoft SharePoint Online",
    "Type": "cloud-application"
  },
  {
    "$id": "4",
    "Name": "bob",
    "UPNSuffix": "jingtoso.com",
    "AadUserId": "6836ed6d-2a15-4889-ad24-fbda46b5ffda",
    "Type": "account"
  }
]
Detection


[
  {
    "Href"